<a href="https://colab.research.google.com/github/VandadII/colab-notebooks/blob/main/Forge_SDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, subprocess, time, glob
#@title <font size="6" color="orange">Stable diffusion Forge</font>
#@markdown ####ForgeUI
#@markdown See version names <a href="https://github.com/lllyasviel/stable-diffusion-webui-forge/commits/main/" target="_blank">here</a>. Leave it blank for the latest.
output_path = 'AI_PICS' #@param {type:"string"}
username='a' #@param {type:"string"}
password='a' #@param {type:"string"}
version='f53307881bfd824dbdce6ac0d4bba04d9a74ab36' #@param {type: "string"}

#@markdown ## Only check the models you are going to use:
#@markdown ### Flux models
Flux1_dev = True #@param{type: "boolean"}
Flux1_schnell = False #@param{type: "boolean"}

#@markdown ### SDXL models
SDXL_1 = False #@param{type: "boolean"}
JuggernautXL_v8 = False #@param{type: "boolean"}
Pony_Diffusion_XL_v6 = False #@param{type: "boolean"}

#@markdown ### v1.5 models:
v1_5_model = False #@param{type: "boolean"}
F222_model = False #@param{type: "boolean"}
Realistic_Vision_model = False #@param{type: "boolean"}
Realistic_Vision_Inpainting_model = False #@param{type: "boolean"}
DreamShaper_model = True #@param{type: "boolean"}
DreamShaper_Inpainting_model = False #@param{type: "boolean"}
OpenJourney_model = False #@param{type: "boolean"}
Anything_v3_model = False #@param{type: "boolean"}
Inkpunk_Diffusion_model = False #@param{type: "boolean"}

#@markdown ### ControlNet models:
SD_1_5_ControlNet_models = False #@param{type: "boolean"}
SDXL_ControlNet_models = False #@param{type: "boolean"}
IP_Adapter_models = False #@param{type: "boolean"}

#@markdown ### Extensions:
Aspect_Ratio_Helper = True #@param{type: "boolean"}
Infinite_Image_Browser = True #@param{type: "boolean"}

#@markdown ### Install models from URL (Huggingface or Civitai. Right-click download button and copy link. Separate multiple links with comma).
Checkpoint_models_from_URL = '' #@param {type: "string"}
#@markdown For Checkpoint_models_from_URL, use full model checkpoints (e.g., https://civitai.com/api/download/models/288982 for JuggernautXL). Appears in "Checkpoint" section.
LoRA_models_from_URL = 'https://civitai.com/api/download/models/1077912?type=Model&format=SafeTensor,https://civitai.green/api/download/models/993999?type=Model&format=SafeTensor,https://civitai.green/api/download/models/1026423?type=Model&format=SafeTensor' #@param {type: "string"}
#@markdown For LoRA_models_from_URL, use LoRA models (e.g., https://civitai.com/api/download/models/1077912). Appears in "LoRA" section. Use commas for multiple URLs.
Civitai_API_Key = '1cdc555856a2aad08d5bb54da04bbd53' #@param {type: "string"}
#@markdown Create API key [here](https://civitai.com/user/account). Required for private Civitai models.

#@markdown ### Install extensions from URL (separate them with comma).
Extensions_from_URL = '' #@param {type: "string"}

#@markdown ### Extra Web-UI arguments
Extra_arguments = '' #@param {type: "string"}

Clear_Output = False #@param{type: "boolean"}

def clear():
    from IPython.display import clear_output; return clear_output()

def fetch_bytes(url_or_path):
    if str(url_or_path).startswith('http://') or str(url_or_path).startswith('https://'):
        from urllib.request import urlopen
        return urlopen(url_or_path)
    return open(url_or_path, 'r')

def packages():
    import sys, subprocess
    return [r.decode().split('==')[0] for r in subprocess.check_output([sys.executable, '-m', 'pip', 'freeze']).split()]

def downloadModel(url, rename = None):
    url = url.strip()
    url = url.rstrip('/')
    if 'huggingface.co' in url:
        if rename:
            filename = rename
        else:
            filename = url.split('/')[-1]
            filename = filename.removesuffix('?download=true')
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -o {filename}
    else:
        # civitai
        if rename:
            if '?' in url:
                !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}&token={Civitai_API_Key}" -o {rename}
            else:
                !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}?token={Civitai_API_Key}" -o {rename}
        else:
            if '?' in url:
                !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}&token={Civitai_API_Key}"
            else:
                !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}?token={Civitai_API_Key}"

def download_models():
    # Download checkpoints
    %cd {root}/stable-diffusion-webui-forge/models/Stable-diffusion
    print(f'⏳ Downloading checkpoints to {root}/stable-diffusion-webui-forge/models/Stable-diffusion ...')
    # Flux
    if Flux1_dev:
        downloadModel('https://huggingface.co/lllyasviel/flux1-dev-bnb-nf4/resolve/main/flux1-dev-bnb-nf4-v2.safetensors')
    if Flux1_schnell:
        downloadModel('https://huggingface.co/silveroxides/flux1-nf4-weights/resolve/main/flux1-schnell-bnb-nf4.safetensors')

    # SD 1.5
    if v1_5_model:
        downloadModel('https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors')
    if F222_model:
        downloadModel('https://huggingface.co/acheong08/f222/resolve/main/f222.ckpt')

    if Realistic_Vision_model:
        downloadModel('https://huggingface.co/SG161222/Realistic_Vision_V5.1_noVAE/resolve/main/Realistic_Vision_V5.1_fp16-no-ema.safetensors')
    if Realistic_Vision_Inpainting_model:
        downloadModel('https://huggingface.co/SG161222/Realistic_Vision_V5.1_noVAE/resolve/main/Realistic_Vision_V5.1-inpainting.safetensors')

    if DreamShaper_model:
        downloadModel('https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_pruned.safetensors')
    if DreamShaper_Inpainting_model:
        downloadModel('https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_INPAINTING.inpainting.safetensors')

    if OpenJourney_model:
        downloadModel('https://huggingface.co/prompthero/openjourney/resolve/main/mdjrny-v4.ckpt')
    if Anything_v3_model:
        downloadModel('https://huggingface.co/admruul/anything-v3.0/resolve/main/Anything-V3.0-pruned-fp16.safetensors')
    if Inkpunk_Diffusion_model:
        downloadModel('https://huggingface.co/Envvi/Inkpunk-Diffusion/resolve/main/Inkpunk-Diffusion-v2.ckpt')

    # SDXL
    if SDXL_1:
        downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors')
        downloadModel('https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors')

    if JuggernautXL_v8:
        downloadModel('https://civitai.com/api/download/models/288982')

    if Pony_Diffusion_XL_v6:
        downloadModel('https://huggingface.co/Magamanny/Pony-Diffusion-V6-XL/resolve/main/ponyDiffusionV6XL_v6StartWithThisOne.safetensors')

    if Checkpoint_models_from_URL:
        for m in Checkpoint_models_from_URL.split(','):
            print(f'Downloading checkpoint: {m}')
            downloadModel(m)

    # Download LoRAs
    !mkdir -p {root}/stable-diffusion-webui-forge/models/Lora
    %cd {root}/stable-diffusion-webui-forge/models/Lora
    print(f'⏳ Downloading LoRAs to {root}/stable-diffusion-webui-forge/models/Lora ...')
    if LoRA_models_from_URL:
        for m in LoRA_models_from_URL.split(','):
            print(f'Downloading LoRA: {m}')
            downloadModel(m)

    # Download VAEs (only for SD 1.5 models, skip for Flux.1)
    if any([v1_5_model, F222_model, Realistic_Vision_model, Realistic_Vision_Inpainting_model, DreamShaper_model, DreamShaper_Inpainting_model, OpenJourney_model, Anything_v3_model, Inkpunk_Diffusion_model]):
        !mkdir -p {root}/stable-diffusion-webui-forge/models/VAE
        %cd {root}/stable-diffusion-webui-forge/models/VAE
        print(f'⏳ Downloading VAEs to {root}/stable-diffusion-webui-forge/models/VAE ...')
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt -d {root}/stable-diffusion-webui-forge/models/VAE/ -o vae-ft-ema-560000-ema-pruned.ckpt
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {root}/stable-diffusion-webui-forge/models/VAE/ -o vae-ft-mse-840000-ema-pruned.ckpt

    # List downloaded files to verify
    print('Checkpoint files:')
    !ls -l {root}/stable-diffusion-webui-forge/models/Stable-diffusion
    print('LoRA files:')
    !ls -l {root}/stable-diffusion-webui-forge/models/Lora
    if any([v1_5_model, F222_model, Realistic_Vision_model, Realistic_Vision_Inpainting_model, DreamShaper_model, DreamShaper_Inpainting_model, OpenJourney_model, Anything_v3_model, Inkpunk_Diffusion_model]):
        print('VAE files:')
        !ls -l {root}/stable-diffusion-webui-forge/models/VAE

    # Optional: Move misplaced LoRA files from Stable-diffusion to Lora (uncomment if LoRAs appear in Checkpoint section)
    # !find {root}/stable-diffusion-webui-forge/models/Stable-diffusion -name "*.safetensors" -not -name "*flux1*" -not -name "*DreamShaper*" -exec mv {} {root}/stable-diffusion-webui-forge/models/Lora/ \;

def updatePython():
    !python --version > /content/pyversion
    with open('/content/pyversion', 'r') as file:
        if '3.10' in file.read():
            print('Python 3.10 detected. Skip installation.')
            return

    #install python 3.10
    !apt-get update -y
    !apt-get install python3.10

    #change alternatives
    !rm /usr/local/bin/python
    !rm /usr/local/bin/pip
    !sudo apt-get install python3.10-distutils
    !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
    !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py

def link_files(source, dest):
    '''Create symlinks for all files in the source folder to the dest folder
    Args:
        source: Absolute path of the source folder
        dest: Absolute path of the destination folder
    '''
    if not os.path.exists(source):
        !mkdir -p {source}
    if not os.path.exists(dest):
        !mkdir -p {dest}
    model_files = glob.glob(source + '/*')
    %cd {dest}
    for f in model_files:
        print(f'Linking model {f} in {dest}')
        !ln -s {f}

def initLocalStorage(output_path, forge_local_path):
    # output_path: E.g. /content/AI_PICS
    # forge_local_path: e.g. /content/stable-diffusion-webui-forge
    forge_path = f"{output_path}/Forge"

    if not os.path.exists(output_path):
        !mkdir -p {output_path}

    if not os.path.exists(forge_path):
        !mkdir -p {forge_path}

    # Use config files in local storage
    # ui-config.json
    if not os.path.exists(forge_path + '/ui-config.json'):
        print("Create new ui-config.json file.")
        !wget https://github.com/sagiodev/stablediffusion_webui/raw/master/ui-config.json -O {forge_path + '/ui-config.json'}
    !ln -s {forge_path + '/ui-config.json'} {forge_local_path}

    # Config.json
    if not os.path.exists(forge_path + '/config.json'):
        print("Create new config.json file.")
        !wget https://github.com/sagiodev/stablediffusion_webui/raw/master/config.json -O {forge_path + '/config.json'}
    !ln -s {forge_path + '/config.json'} {forge_local_path}

    # styles.csv
    if not os.path.exists(forge_path + '/styles.csv'):
        print("Create new styles.csv file.")
        !wget https://raw.githubusercontent.com/sagiodev/stablediffusion_webui/master/styles.csv -O {forge_path + '/styles.csv'}
    !ln -s {forge_path + '/styles.csv'} {forge_local_path}

    # save parameter file in local storage
    %cd {forge_local_path}
    if not os.path.exists(forge_path + '/params.txt'):
        !touch {forge_path + '/params.txt'}
    !ln -s {forge_path}/params.txt

    # create image output folder
    %cd {forge_local_path}
    if not os.path.exists(f'{output_path}/outputs'):
        !mkdir -p {output_path}/outputs
    !ln -s {output_path}/outputs

    # create extension folder
    %cd {forge_local_path}
    if not os.path.exists(f'{forge_path}/extensions'):
        !mkdir -p {forge_path}/extensions
    if os.path.exists(f'extensions'):
        !rm -rf extensions
    !ln -s {forge_path}/extensions

def installControlNetModels():
    !mkdir -p {root}/stable-diffusion-webui-forge/models/ControlNet
    %cd {root}/stable-diffusion-webui-forge/models/ControlNet
    print(f'⏳ Downloading ControlNet models to {root}/stable-diffusion-webui-forge/models/ControlNet ...')
    if SD_1_5_ControlNet_models:
        downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth')
        downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth')
        downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth')
        downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth')
        downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth')
        downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth')
        downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth')
        downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth')
        downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth')
        downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth')
        downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth')
        downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth')
        downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth')
        downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth')
        downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth')
        downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth')
        downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.safetensors')
        downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/v2/control_v1p_sd15_qrcode_monster_v2.safetensors')

    if SDXL_ControlNet_models:
        downloadModel('https://huggingface.co/xinsir/controlnet-openpose-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors', rename='diffusion_xl_openpose.safetensors')
        downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_full.safetensors')
        downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_mid.safetensors')
        downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_xl.pth')
        downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur.safetensors')
        downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur_anime.safetensors')
        downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_scribble_anime.safetensors')
        downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_256lora.safetensors')
        downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors')
        downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors')

    if IP_Adapter_models:
        downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors')
        downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors')
        downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors')
        downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin')
        downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.safetensors')
        downloadModel('https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl.bin')
        !pip install insightface

def installExtensionsFromURL(urls):
    for url in urls.split(','):
        print("Cloning extension from URL: %s"%url)
        %cd {extension_root}
        !git clone {url}
        # delete the .git directory
        !cd -- {url.rstrip('/').split('/')[-1]};rm -rf .git

def searchAndReplace(filePath, orignalStr, newStr):
    orignalStr = orignalStr.replace('/', '\/')
    newStr = newStr.replace('/', '\/')
    !sed -i 's/{orignalStr}/{newStr}/g' {filePath}

def install_Infinite_Image_Browser():
    installExtensionsFromURL('https://github.com/zanllp/sd-webui-infinite-image-browsing')
    %cd {extension_root}/sd-webui-infinite-image-browsing
    !sed 's/IIB_SECRET_KEY=/IIB_SECRET_KEY=SDA/' .env.example > .env

# Define root and extension paths
root = '/content/'
extension_root = f'{root}/stable-diffusion-webui-forge/extensions'

# Update to python 3.10
updatePython()

# Clone Forge
os.chdir(root)
!apt-get -y install -qq aria2
!git clone https://github.com/lllyasviel/stable-diffusion-webui-forge

# Forge first launch
%cd {root}/stable-diffusion-webui-forge
!git checkout -f {version}
!COMMANDLINE_ARGS="--exit"  python launch.py

# Setup local storage folders
output_path = '/content/' + output_path
initLocalStorage(output_path, f"{root}/stable-diffusion-webui-forge")

if Aspect_Ratio_Helper:
    installExtensionsFromURL('https://github.com/altoiddealer/--sd-webui-ar-plusplus')

if Infinite_Image_Browser:
    install_Infinite_Image_Browser()

if Extensions_from_URL:
    installExtensionsFromURL(Extensions_from_URL)

# downgrade httpx to avoid TypeError: AsyncConnectionPool.__init__() got an unexpected keyword argument 'socket_options'
!pip install httpx==0.24.1

# fix matplotlib backend ValueError
!python -m pip install matplotlib-inline

# Models
download_models()
installControlNetModels()

# clear output
if Clear_Output:
    clear()

# Launch Forge with Gradio sharing
%cd {root}/stable-diffusion-webui-forge
args = f'--gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue --share'

if username and password:
    args += f' --gradio-auth "{username}":"{password}" '
args += ' ' + Extra_arguments
print(f'WEBUI ARGUMENTS: {args}')
print('Note: In the Forge UI, select "flux1-dev-bnb-nf4-v2" for Flux.1 or "DreamShaper_8_pruned" for DreamShaper. Ensure LoRAs are compatible with the selected model.')
!python {root}/stable-diffusion-webui-forge/launch.py {args}

# Optional: Download generated outputs (uncomment to enable)
# from google.colab import files
# import glob
# for file in glob.glob('/content/AI_PICS/outputs/*'):
#     files.download(file)